# Génération des réseaux de formes à partir des règles
- Swim1 basé sur l'échantillon initial
 - Gén-1 : génération des formes d'après les contextes phonologiques
 - Gén-2 : génération du réseau d'après les contextes phonologiques
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Swim2 basé sur le réseau calculé par Swim1
 - Exp : génération d'un nouvel échantillon basé sur Swim1
 - Gén-1 : génération des formes sans contexte phonologique
 - Gén-2 : génération du réseau sans contexte phonologique
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Évaluation  

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [5]:
# -*- coding: utf8 -*-
import codecs,operator,datetime,os,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [6]:
%matplotlib inline

In [7]:
import yaml

In [8]:
from ipywidgets import FloatProgress
from IPython.display import display, HTML

In [9]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [10]:
saut="\n"

### Préparation des matrices de traits

In [11]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

# Choix de l'échantillon et des règles
- *sampleFile* est le nom de l'échantillon de départ
- *rulesFile* est le nom du fichier de règles
- *goldFile* est le nom du lexique Gold de référence

In [34]:
repFiles="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/"
sampleFile="IMM21-00-X-Morphomes.csv"
tirageFile="IMM21-00.pkl"
tirageFile="IMM21-00-X-Morphomes.pkl"
rulesFile="IMM21-00-X-Morphomes-Regles.pkl"
repGold="/Volumes/BroadExt/ownCloud/Recherche/Boye/HDR/Data/Samples/"
goldFile="MGC-171229-Verbes3.pkl"
fSample=repFiles+sampleFile
fInitial=fSample
fTirage=repFiles+tirageFile
fRules=repFiles+rulesFile
fGold=repGold+goldFile

In [13]:
sampleType="-X"
casesType="-Morphomes"
listeFormesOutput=["FS","FP"]

In [14]:
genDigraphe=False
genGraphe=False
genFormeVotes=True
genCliques=True
plotDistributionCliques=False

In [15]:
with open(fTirage, 'rb') as input:
    sampleTirage = pickle.load(input)
sampleTirage.head()

,lexeme,phono,case,morphome,freq,tir1
0,abaisser,abEs,pI2S,pI2S,1,1
1,abaisser,abEs,pi1S,pi1S,1,1
2,abaisser,abEs,pi2S,pi2S/pi3S,2,2
3,abaisser,abEs,pi3P,pi3P,1,1
4,abaisser,abEs6rE,fi1S,fi1S/pc1S/pc2S/pc3P/pc3S,4,4


In [16]:
if not "morphome" in sampleTirage.columns.tolist():
    sampleTirage["morphome"]=sampleTirage["case"]
morphomeCases=sampleTirage[["case","morphome"]].drop_duplicates().to_dict()
# morphomeCases

In [25]:
casesMC=morphomeCases["case"]
morphomesMC=morphomeCases["morphome"]
dictMorphomeCases={}
for element in casesMC:
    dictMorphomeCases[casesMC[element]]=morphomesMC[element].split("/")
dictMorphomeCases

{u'ai1P': [u'ai1P'],
 u'ai1S': [u'ai1S'],
 u'ai2P': [u'ai2P'],
 u'ai2S': [u'ai2S', u'ai3S', u'is3S'],
 u'ai3P': [u'ai3P'],
 u'fi1P': [u'fi1P', u'fi3P'],
 u'fi1S': [u'fi1S', u'pc1S', u'pc2S', u'pc3P', u'pc3S'],
 u'fi2P': [u'fi2P'],
 u'fi2S': [u'fi2S', u'fi3S'],
 u'ii1P': [u'ii1P'],
 u'ii1S': [u'ii1S', u'ii2S', u'ii3P', u'ii3S'],
 u'ii2P': [u'ii2P'],
 u'inf': [u'inf'],
 u'is1P': [u'is1P', u'ppFP', u'ppFS'],
 u'is1S': [u'is1S', u'is2S', u'is3P'],
 u'is2P': [u'is2P'],
 u'pI1P': [u'pI1P'],
 u'pI2P': [u'pI2P'],
 u'pI2S': [u'pI2S'],
 u'pP': [u'pP'],
 u'pc1P': [u'pc1P'],
 u'pc2P': [u'pc2P'],
 u'pi1P': [u'pi1P'],
 u'pi1S': [u'pi1S'],
 u'pi2P': [u'pi2P'],
 u'pi2S': [u'pi2S', u'pi3S'],
 u'pi3P': [u'pi3P'],
 u'ppMP': [u'ppMP', u'ppMS'],
 u'ps1P': [u'ps1P'],
 u'ps1S': [u'ps1S', u'ps2S', u'ps3P', u'ps3S'],
 u'ps2P': [u'ps2P']}

### Choix original

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [18]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [19]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [20]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [21]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [26]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=True):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [27]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [28]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


# Lecture de l'échantillon

In [29]:
phonologicalMap=sampleType.strip("-")
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [30]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

### Vérification de la phonotactique des glides du français
- si *prononciation* est *None* renvoyer *None*
- ajout de diérèses dans les séquences mal-formées
- vérification des séquences consonne+glide à la finale

In [31]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [32]:
def checkFrench(prononciation):
    if prononciation:
        result=recoder(prononciation)
        m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
        if m:
            print ("pb avec un glide final", prononciation)
        m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
        if m:
            n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
            if not n:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
        m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
        if m:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
        result=result.replace("Jj","J")
    else:
        result=prononciation
    return result

In [35]:
paradigmes=pd.read_csv(fSample,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [36]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [37]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

In [38]:
paradigmes.stack().value_counts(dropna=True).sum()

43497

In [39]:
len(paradigmes.dropna(thresh=1)["lexeme"])

4803

## Préparation des données initiales et de référence

In [40]:
listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
#listeTest=[u"asseoir",u"balayer",u"manger"]
nbVerbes=len(listeTest)
print (nbVerbes)

4803


### Préparation des formes initiales

In [41]:
initialParadigmes=pd.read_csv(fInitial,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')

In [42]:
initialForms=pd.melt(initialParadigmes[paradigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()

initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)

initialForms.set_index(["lexeme-case"],inplace=True)

initialFormsIndex=initialForms.index.tolist()

### Préparation des formes de référence

In [43]:
#with open(goldPrefix+"/"+goldFile,"rb") as input:

# with open(goldPrefix+goldFile,"rb") as input:
#     lexiqueGold=pickle.load(input)
lexiqueGold=pd.read_pickle(path=fGold)

'''Rectifications phonologiques'''
lexiqueGold["phono"]=lexiqueGold["phono"].apply(lambda x: checkFrench(x))
completeParadigmes=pd.pivot_table(lexiqueGold, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

'''Mise en liste des formes de références'''
goldTestForms=pd.melt(completeParadigmes[completeParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
goldTestForms["lexeme-case"]=goldTestForms["lexeme"]+"-"+goldTestForms["case"]
goldTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
goldTestForms.set_index(["lexeme-case"],inplace=True)

'''Extraction des formes de références pertinentes'''
goldForms=goldTestForms.loc[~goldTestForms.index.isin(initialFormsIndex)]
goldFormsIndex=goldForms.index.tolist()

# Lecture des règles

In [44]:
with open(fRules, 'rb') as input:
    resultatsLecture = pickle.load(input)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [45]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(goldCases):
    print ("Attention l'analyse ne comprend pas toutes les cases")
    print (sorted(analyseCases))
    print (sorted(goldCases))

# Préparations pour la génération des formes

In [46]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [47]:
def generateForms(lexeme,contextFree=False):
    # pb avec l'encodage des noms de lexèmes
    candidats=paradigmeDistribution(lexeme)
    bLexParadigme=paradigmes["lexeme"].str.contains("^"+lexeme+"$")
    casesSamples=paradigmes[bLexParadigme].columns[paradigmes[bLexParadigme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[bLexParadigme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element,contextFree),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart,contextFree))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [48]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        # modif pour networkx v2
        # coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        coefGraphe=float(digraphe.adj[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [49]:
def generateParadigms(generation1,genDigraphe=True,contextFree=False):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart,contextFree)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

# Génération d'un jeu de cliques

In [50]:
def generate(lexeme,genDigraphe=True,contextFree=False):
    # print (lexeme,end=", ")
    generation1=generateForms(lexeme,contextFree)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe,contextFree)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [51]:
paradigmes.dropna(thresh=1).count().sum()-paradigmes.dropna(thresh=1)["lexeme"].count()

38694

#### Calculer le score de la clique

In [52]:
def cliqueScore(clique,graph):
    score=0
    if len(clique)>1:
        for (depart,arrivee) in it.combinations_with_replacement(clique,2):
            score+=graph[depart][arrivee]["weight"]
    return score

In [53]:
def splitArrivee(arrivee):
    arriveeMorceaux=arrivee.split("-")
    if len(arriveeMorceaux)<3:
        print (arrivee,arriveeMorceaux)
    lexeme="-".join(arriveeMorceaux[:-2])
    formeArrivee=arriveeMorceaux[-2]
    caseArrivee=arriveeMorceaux[-1]
    return (lexeme,formeArrivee,caseArrivee)
    
# trouver tous les liens vers FS-* et FP-*
# regrouper par forme 
# calculer les proportions
# renvoyer les proportions par forme
# avec le nombre de forme à l'appui
def formeScore(forme,graph):
    scores={}
    scoresNormes={}
    # modif pour networkx v2
    # for depart in graph.edge edge=>adj:
    for depart in graph.adj:
        for arrivee in graph.adj[depart]:
            (lexeme, formeArrivee, caseArrivee)=splitArrivee(arrivee)
            if caseArrivee==forme:
#                print (depart, formeArrivee, graph.edge[depart][arrivee])
                if not formeArrivee in scores:
                    scores[formeArrivee]=0
                scores[formeArrivee]+=graph.adj[depart][arrivee]["weight"]
    totalArrivee=0
    for formeArrivee in scores:
        totalArrivee+=scores[formeArrivee]
    for formeArrivee in scores:
        scoresNormes[formeArrivee]=scores[formeArrivee]/totalArrivee
    return (scores,scoresNormes)
        

## Préparations pour SWIM

In [54]:
def generateAnalysis(globDigraphe,globGraphe,contextFree=False):
    numClique=0
    progressBar = FloatProgress(min=0, max=nbVerbes-1,description="Generation (%d verbs)"%nbVerbes)
    display(progressBar)
    for i,element in enumerate(listeTest):
        cliquesScores[element]={}
        cliquesListes[element]={}
        if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
        progressBar.value=i
        #print (element)
        result=generate(element,genDigraphe,contextFree)
        (generation,lexDigraphe,lexGraphe,lexCliques)= result
    #    print (generation,lexDigraphe,lexGraphe,lexCliques)
        if genFormeVotes:
            formesScores[element]={}
            formesScoresNormes[element]={}
            for formeOutput in listeFormesOutput:
                (formesScores[element][formeOutput],formesScoresNormes[element][formeOutput])=formeScore(formeOutput,lexDigraphe)
        if genDigraphe:
            globDigraphe=nx.union(globDigraphe,lexDigraphe)
        if genGraphe:
            globGraphe=nx.union(globGraphe,lexGraphe)
        cliques.extend(lexCliques)
        for clique in lexCliques:
            cliquesScores[element][numClique]=cliqueScore(clique,lexGraphe)
            cliquesListes[element][numClique]=clique
            numClique+=1
    return globDigraphe,globGraphe,numClique

In [55]:
def dictCliqueForms(clique):
    result={}
    for element in clique:
        lexeme,forme,case=splitArrivee(element)
        for c in dictMorphomeCases[case]:
            result[c]=forme
    return result

def dictPdRowForms(row):
    result={}
    for case in sampleCases:
        print (case,row[case].values[0])
    return result

def tableZero(case):
    if case in sampleCases:
        return u"Ø"
    else:
        return u"="

def makeTable(dictForms,title=""):
    tabular=[]
    labelTenseCode={"pi":"Present","ii":"Imperfective","ai":"Simple Past","fi":"Future",
                    "ps":"Subjunctive Pres.","is":"Subjunctive Imp.","pc":"Conditional","pI":"Imperative",
                    "inf":"Infinitive",
                    "ppMS":"Past Part. MS","ppMP":"Past Part. MP",
                    "ppFS":"Past Part. FS","ppMP":"Past Part. FP"
                   }
    def makeLine6(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if (case in dictForms) and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"

    def makeLine3(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                    line.append(r"<td>%s</td>"%(dictForms[case]))
                else:
                    line.append(r"<td>%s</td>"%(tableZero(case)))
            else:
                line.append(r"<td>%s</td>"%(u"---"))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
    def makeLineNF():
        line=[]
        line.append(r"<th>%s</th>"%"NF")
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
        
    top=[
        r"<table>",
        r"<caption style='caption-side:bottom;text-align:center'>",
        "Verbe : %s"%title,
        r"</caption>",
#        r"<tr><th/><th>1S</th><th>2S</th><th>3S</th><th>1P</th><th>2P</th><th>3P</th></tr>"
        r"<tr><th/><th>1SG</th><th>2SG</th><th>3SG</th><th>1PL</th><th>2PL</th><th>3PL</th></tr>"
        ]
    bottom=[
        r"</table>"
        ]
    tabular.append("\n".join(top))
    for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
        tabular.append(makeLine6(tenseCode))
    tabular.append(makeLine3("pI"))
    tabular.append(makeLineNF())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)    

def diffParadigme(lexeme):
    outLen=lexemeMaxCliques[lexeme][0]
    bLexParadigme=paradigmes["lexeme"].str.contains("^"+lexeme+"$")
    inLen=paradigmes[bLexParadigme].notnull().sum(axis=1).values[0]-1
    if outLen>inLen:
        print (lexemeMaxCliques[lexeme][1])
        print (paradigmes[paradigmes["lexeme"]=="grandir"].values)
    return outLen-inLen
    

In [56]:
def checkFidelite(fidelite,clique):
    lFidele=False
    for element in clique:
        if fidelite in element:
            lFidele=True
    return lFidele

def generateCliques(contextFree=False):
    
    def bruteCliques(lexeme,maxCliqueSize=51):
        cliquesBrutes={n+1:0 for n in range(maxCliqueSize)}
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                if not longueur in cliquesBrutes:
                    cliquesBrutes[longueur]=0
                cliquesBrutes[longueur]+=1
        return cliquesBrutes
    
    globDigraphe=nx.DiGraph()
    globGraphe=nx.Graph()

    globDigraphe,globGraphe,numClique=generateAnalysis(globDigraphe,globGraphe,contextFree)
    print 

    lexemeMaxCliques={}
    lexemeParadigmes={}
    progressBarCliques = FloatProgress(min=0, max=len(cliquesListes)-1,description="Analysis (%d verbes)"%len(cliquesListes))
    display(progressBarCliques)
    for lexeme in cliquesListes:
        progressBarCliques.value+=1
        maxLen=max([len(c) for c in cliquesListes[lexeme].values()])
        lexemeMaxCliques[lexeme]=bruteCliques(lexeme,maxLen)
        print (lexeme,"Nombre de cliques",sum([v for k,v in lexemeMaxCliques[lexeme].iteritems()]))
        maxNbCliques=max([v for k,v in lexemeMaxCliques[lexeme].iteritems()])
        if plotDistributionCliques:
            ax=pd.DataFrame.from_dict(lexemeMaxCliques[lexeme],orient="index").plot(kind="bar",legend=False,grid=True,figsize=(10,3))
            ax.set(xlim=(0,maxLen+.5),ylim=(0,maxNbCliques+10))
            ax.set_xlabel("Clique Size in Cells",fontsize=16)
            ax.set_ylabel("Number of Cliques",fontsize=16)

        dictParadigmes=paradigmes.set_index("lexeme").to_dict(orient="index")

        cliquesFideles={}
        fidelites=[v+"-"+k for k,v in dictParadigmes[lexeme].iteritems() if isinstance(v,unicode)]
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                fidele=True
                for fidelite in fidelites:
                    if "," in fidelite:
                        fideliteForme,fideliteCase=fidelite.split("-")
                        fideliteFormes=fideliteForme.split(",")
                        fideliteItems=[fideliteF+"-"+fideliteCase for fideliteF in fideliteFormes]
#                        print (fideliteItems)
                        lFidele=True
                        for f in fideliteItems:
                            lFidele=lFidele & checkFidelite(f,l)
                    else:
                        lFidele=checkFidelite(fidelite,l)
                    if not lFidele:
                        fidele=False
                        break
                if fidele:
                    if not longueur in cliquesFideles:
                        cliquesFideles[longueur]=[]
                    cliquesFideles[longueur].append(l)
#                else:
#                    if lexeme==u"bégayer": print ()
#        print ([(k,len(v)) for k,v in cliquesFideles.iteritems()])
        if cliquesFideles:
            maxCliquesCard=max([k for k,v in cliquesFideles.iteritems()])
    #        print (maxCliquesCard)
    #        print (cliquesScores[lexeme])
    #        print (cliquesListes[lexeme])
            lexemeParadigmes[lexeme]=[]
            maxScoreCliques=max([clique for cliqueNumber, clique in cliquesScores[lexeme].items()])
            maxCardScoreNums=[numC for numC, c in cliquesListes[lexeme].items() if c in cliquesFideles[maxCliquesCard]]
            maxCardScore=max([scoreC for numC, scoreC in cliquesScores[lexeme].items() if numC in maxCardScoreNums])
    #        print ("max score among all cliques:",maxScoreCliques)
            print ("max score among faithfull cliques of %d forms:"%maxCliquesCard,maxCardScore)
            for c in cliquesFideles[maxCliquesCard]:
                cNumber=[cliqueNumber for cliqueNumber, clique in cliquesListes[lexeme].items() if clique == c]
                if len(cNumber)!=1:
                    print ("TOO MANY SCORES PROBLEM WITH CLIQUE", cNumber)
    #            print ("Liste n°",cNumber[0],cliquesScores[lexeme][cNumber[0]])
    #            print (sorted(cliquesScores[lexeme].items(), key=operator.itemgetter(1)))
    #            display(HTML(makeTable(dictCliqueForms(c),title=c[0].split("-")[0])))
    #            print (cliquesScores[lexeme][cNumber[0]], maxCardScore)
                if cliquesScores[lexeme][cNumber[0]]==maxCardScore:
                    lexemeParadigmes[lexeme].append(c)
        else:
            lexemeParadigmes[lexeme]=[[lexeme+"-"+f for f in fidelites]]
            print (u"no new faithfull clique, the previous one contained %d forms"%len(lexemeParadigmes[lexeme][0]))
    return lexemeParadigmes

In [57]:
def cutNodeName(nodeName):
    items=nodeName.split("-")
    nbItems=len(items)
    if nbItems>3:
        items=["-".join(items[0:nbItems-2]),items[-2],items[-1]]
    return items

def filledOutClique(cliques):
    if cliques:
        result=[]
        for clique in cliques:
#            print (clique)
            fullClique=[]
            for element in clique:
                if debug: print ("element",element)
                lexeme,forme,case=splitArrivee(element)
                for c in dictMorphomeCases[case]:
                    fullClique.append("-".join([lexeme,forme,c]))
            result.append(fullClique)
        return result
    else:
        return cliques

In [58]:
def extendParadigmes(contextParadigmes,extendMorphomes=False):
    lexemesParadigmeListe=[]
    for lexeme in contextParadigmes:
        if extendMorphomes:
            lexParadigmes=filledOutClique(contextParadigmes[lexeme])
        else:
            lexParadigmes=contextParadigmes[lexeme]
        if len(lexParadigmes)!=1:
            if debug:
                print ("LEXEME WITH A NON UNIQUE PARADIGM PB",len(lexParadigmes),lexeme)
                print (lexParadigmes)
        lexParadigme=lexParadigmes[0]
        for lexForme in lexParadigme:
            lexemesParadigmeListe.append(cutNodeName(lexForme))
    newForms=pd.DataFrame(lexemesParadigmeListe)
    newForms.columns=["lexeme","form","case"]
#    newParadigmes=newForms.pivot(index="lexeme", columns="case", values="form")
    newParadigmes=pd.pivot_table(newForms, values='form', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    for i in newParadigmes.itertuples():
#        print (i[0],i[1])
        lexeme=i[1]
        bLexParadigme=paradigmes["lexeme"].str.contains("^"+lexeme+"$")
        lexemeIndexes=paradigmes.lexeme[bLexParadigme].index.tolist()
        if lexemeIndexes:
            lexemeIndex=lexemeIndexes[0]
        else:
            print (i,lexeme,lexemeIndexes)
        bLexNewParadigme=newParadigmes["lexeme"].str.contains("^"+lexeme+"$")
        newParadigmes.loc[bLexNewParadigme,"index"]=int(lexemeIndex)
    newParadigmes.set_index("index",inplace=True)   
    return paradigmes.combine_first(newParadigmes)

In [59]:
def countSplits(dfForms):
    dfForms.loc[:,"split"]=dfForms.loc[:,"value"].str.split(",")
    return dfForms["split"].str.len().sum()

def calculerResultats(contextParadigmes,extension="-Swim2"):
    
    '''Préparer le paradigme des prédictions'''
    brutParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=False)
    finalParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=True)
    finalParadigmes.to_csv(fSample.replace(".csv","")+"%s.csv"%extension,encoding="utf8",sep=";")
    finalTestForms=pd.melt(finalParadigmes[finalParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
    finalTestForms["lexeme-case"]=finalTestForms["lexeme"]+"-"+finalTestForms["variable"]
    finalTestForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
    print (finalTestForms.columns)
    # finalTestForms["lexeme-case"]=finalTestForms["lexeme"]+"-"+finalTestForms["case"]
    # finalTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
    finalTestForms.set_index(["lexeme-case"],inplace=True)
    
    '''Soustraire les formes initiales'''
    finalForms=finalTestForms.loc[~finalTestForms.index.isin(initialFormsIndex)]
    finalFormsIndex=finalForms.index.tolist()
    
    '''Calculer les sur/sous-générations'''
    underGeneration=goldForms.loc[~goldForms.index.isin(finalFormsIndex)]
    overGeneration=finalForms.loc[~finalForms.index.isin(goldFormsIndex)]
    
    '''Réduire les prédictions et la référence aux cases communes'''
    predictedForms=finalForms.loc[finalForms.index.isin(goldFormsIndex)]
    actualForms=goldForms.loc[goldForms.index.isin(finalFormsIndex)]
    
    '''Créer un tableau pour les comparaisons'''
    compareForms=predictedForms.copy()
    compareForms.loc[:,"right"]=actualForms.loc[:,"value"]
    
    '''Séparer les cases identiques des cases différentes'''
    sameForms=compareForms[compareForms["value"]==compareForms["right"]]
    diffForms=compareForms[compareForms["value"]!=compareForms["right"]]

    
    '''Sauvegarder les comparatifs'''
    overGeneration.to_csv(fSample.replace(".csv","")+"-overGeneration%s.csv"%extension,encoding="utf8")
    underGeneration.to_csv(fSample.replace(".csv","")+"-underGeneration%s.csv"%extension,encoding="utf8")
    sameForms.to_csv(fSample.replace(".csv","")+"-sameForms%s.csv"%extension,encoding="utf8")
    diffForms.to_csv(fSample.replace(".csv","")+"-diffForms%s.csv"%extension,encoding="utf8")
    
    
    '''Transformer les surabondances en liste'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"value"].str.split(",")
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"right"].str.split(",")

    '''Transformer les surabondances en set()'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"split-value"].apply(set)
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"split-right"].apply(set)
    
    '''Calculer le nombre de formes (y compris surabondances)'''
    nbValues=diffForms["split-value"].str.len().sum()
    nbRights=diffForms["split-right"].str.len().sum()

    '''Calculer les identités et les inclusions'''
    nbIdenticalSets=diffForms[diffForms["split-value"]==diffForms["split-right"]]["split-value"].str.len().sum()
    nbIncludedSets=diffForms[diffForms["split-value"]<diffForms["split-right"]]["split-value"].str.len().sum()
    nbWrongForms=(nbValues-nbIdenticalSets-nbIncludedSets)
    underBonus=(nbRights-nbIdenticalSets-nbIncludedSets)

    UG=countSplits(underGeneration)+underBonus
    OG=countSplits(overGeneration)
    TP=countSplits(sameForms)+nbIdenticalSets+nbIncludedSets
    FP=nbWrongForms
    resultCharacteristics=(UG,OG,TP,FP)
    recall=float(TP)/(UG+TP+FP)
    precision=float(TP)/(OG+TP+FP)
    fMeasure=2*recall*precision/(recall+precision)
    resultMeasures=(precision,recall,fMeasure)
    print ("UG",UG ,"OG",OG,"TP",TP,"FP",FP)
    print ("recall", recall, "precision", precision)
    print (fMeasure)
    return (brutParadigmes,finalParadigmes,resultCharacteristics,resultMeasures)


# SWIM1

In [60]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim1ContextParadigmes=generateCliques()

2023-11-03 12:02:14.446640


FloatProgress(value=0.0, description=u'Generation (4803 verbs)', max=4802.0)

0 1102 0, 100 1105 2, 200 1108 4, 300 1112 6, 400 1115 8, 500 1118 10, 600 1120 12, 700 1123 14, 800 1125 16, 900 1129 18, 1000 1132 20, 1100 1135 22, 1200 1138 24, 1300 1141 27, 1400 1143 29, 1500 1146 31, 1600 1149 33, 1700 1152 35, 1800 1154 37, 1900 1158 39, 2000 1201 41, 2100 1204 43, 2200 1207 45, 2300 1210 47, 2400 1212 49, 2500 1216 52, 2600 1218 54, 2700 1221 56, 2800 1224 58, 2900 1227 60, 3000 1230 62, 3100 1233 64, 3200 1236 66, 3300 1239 68, 3400 1242 70, 3500 1246 72, 3600 1249 74, 3700 1252 77, 3800 1255 79, 3900 1258 81, 4000 1301 83, 4100 1304 85, 4200 1307 87, 4300 1310 89, 4400 1313 91, 4500 1316 93, 4600 1319 95, 4700 1322 97, 4800 1325 99, 

FloatProgress(value=0.0, description=u'Analysis (4803 verbes)', max=4802.0)

escamoter Nombre de cliques 9
max score among faithfull cliques of 25 forms: 307.383611761
enquérir Nombre de cliques 78
max score among faithfull cliques of 10 forms: 40.1204884866
réfugier Nombre de cliques 71
max score among faithfull cliques of 14 forms: 78.9501100026
assembler Nombre de cliques 21
max score among faithfull cliques of 23 forms: 265.111293273
décrédibiliser Nombre de cliques 4
max score among faithfull cliques of 24 forms: 281.504095881
ensanglanter Nombre de cliques 7
max score among faithfull cliques of 25 forms: 317.834492425
projeter Nombre de cliques 77
max score among faithfull cliques of 22 forms: 156.250083364
conceptualiser Nombre de cliques 5
max score among faithfull cliques of 24 forms: 281.504095881
forger Nombre de cliques 15
max score among faithfull cliques of 25 forms: 310.042785916
réglementer Nombre de cliques 7
max score among faithfull cliques of 26 forms: 326.945282472
aider Nombre de cliques 24
max score among faithfull cliques of 25 forms: 31

In [61]:
newParadigmes,paradigmesSwim1,characteristicsSwim1,measuresSwim1=calculerResultats(swim1ContextParadigmes,"-Swim1")
%ding

Index([u'value', u'lexeme-case'], dtype='object')


/Users/gilles/opt/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gilles/opt/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/gilles/opt/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

UG 59842 OG 16668 TP 125485 FP 12745
recall 0.633532250899 precision 0.810113752276
0.711023599739


UsageError: Line magic function `%ding` not found.


# SWIM2

## Préparation des paradigmes

In [62]:
paradigmesOriginaux=paradigmes.copy()
paradigmesSample=paradigmesOriginaux[paradigmesOriginaux["lexeme"].isin(listeTest)]

In [63]:
paradigmes=newParadigmes.copy()
paradigmesColumns=paradigmes.columns.tolist()
for c in sampleCases:
    if not c in paradigmesColumns:
        print (c)
        paradigmes[c]=np.NaN

In [64]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim2ContextParadigmes=generateCliques(contextFree=True)

2023-11-03 14:39:13.877329


FloatProgress(value=0.0, description=u'Generation (4803 verbs)', max=4802.0)

0 1339 0, 100 1341 2, 200 1343 4, 300 1345 6, 400 1347 8, 500 1349 10, 600 1351 12, 700 1353 14, 800 1355 16, 900 1356 18, 1000 1359 20, 1100 1400 22, 1200 1403 24, 1300 1405 27, 1400 1407 29, 1500 1408 31, 1600 1410 33, 1700 1412 35, 1800 1414 37, 1900 1416 39, 2000 1418 41, 2100 1420 43, 2200 1422 45, 2300 1424 47, 2400 1426 49, 2500 1428 52, 2600 1430 54, 2700 1432 56, 2800 1434 58, 2900 1436 60, 3000 1438 62, 3100 1440 64, 3200 1442 66, 3300 1444 68, 3400 1446 70, 3500 1448 72, 3600 1450 74, 3700 1452 77, 3800 1454 79, 3900 1456 81, 4000 1458 83, 4100 1500 85, 4200 1501 87, 4300 1503 89, 4400 1505 91, 4500 1507 93, 4600 1510 95, 4700 1511 97, 4800 1514 99, 

FloatProgress(value=0.0, description=u'Analysis (4803 verbes)', max=4802.0)

escamoter Nombre de cliques 988
max score among faithfull cliques of 26 forms: 207.902212309
enquérir Nombre de cliques 1341
max score among faithfull cliques of 15 forms: 19.5269254416
réfugier Nombre de cliques 1236
max score among faithfull cliques of 23 forms: 76.5526228244
assembler Nombre de cliques 1383
max score among faithfull cliques of 25 forms: 172.78941292
décrédibiliser Nombre de cliques 1247
max score among faithfull cliques of 26 forms: 201.386693894
ensanglanter Nombre de cliques 918
max score among faithfull cliques of 26 forms: 218.113081301
projeter Nombre de cliques 1928
max score among faithfull cliques of 26 forms: 156.526024987
conceptualiser Nombre de cliques 1259
max score among faithfull cliques of 26 forms: 204.445678528
forger Nombre de cliques 941
max score among faithfull cliques of 26 forms: 223.017992743
réglementer Nombre de cliques 925
max score among faithfull cliques of 26 forms: 219.42997753
aider Nombre de cliques 1094
max score among faithfull cl

In [65]:
newParadigmes,paradigmesSwim2,characteristicsSwim2,measuresSwim2=calculerResultats(swim2ContextParadigmes,"-Swim2")

Index([u'value', u'lexeme-case'], dtype='object')
UG 52099 OG 17756 TP 133228 FP 15005
recall 0.665036040173 precision 0.802631499678
0.727383906465


In [ ]:
nomFichierResultats=filePrefix+"-X-Resultats.yaml"
if os.path.isfile(nomFichierResultats):
    with open(nomFichierResultats, 'r') as stream:
            resultats=yaml.load(stream)
else:
    resultats={}

if casesType:
    sampleExt=casesType
else:
    sampleExt=sampleType
sampleId=sampleNumber.strip("-")+sampleExt
resultats[sampleId]={}
resultats[sampleId]["Swim1"]={}
resultats[sampleId]["Swim1"]["UG"]=characteristicsSwim1[0]
resultats[sampleId]["Swim1"]["OG"]=characteristicsSwim1[1]
resultats[sampleId]["Swim1"]["TP"]=characteristicsSwim1[2]
resultats[sampleId]["Swim1"]["FP"]=characteristicsSwim1[3]
resultats[sampleId]["Swim1"]["Precision"]=measuresSwim1[0]
resultats[sampleId]["Swim1"]["Recall"]=measuresSwim1[1]
resultats[sampleId]["Swim1"]["F-Measure"]=measuresSwim1[2]
resultats[sampleId]["Swim2"]={}
resultats[sampleId]["Swim2"]["UG"]=characteristicsSwim2[0]
resultats[sampleId]["Swim2"]["OG"]=characteristicsSwim2[1]
resultats[sampleId]["Swim2"]["TP"]=characteristicsSwim2[2]
resultats[sampleId]["Swim2"]["FP"]=characteristicsSwim2[3]
resultats[sampleId]["Swim2"]["Precision"]=measuresSwim2[0]
resultats[sampleId]["Swim2"]["Recall"]=measuresSwim2[1]
resultats[sampleId]["Swim2"]["F-Measure"]=measuresSwim2[2]

yaml.safe_dump(resultats, file(nomFichierResultats, 'w'), encoding='utf-8', allow_unicode=True)

In [ ]:
%ding
%ding

In [ ]:
print ("Sample",sampleNumber.strip("-"))
print ("Swim1",characteristicsSwim1,measuresSwim1)
print ("Swim2",characteristicsSwim2,measuresSwim2)

# Fin du traitement

In [ ]:
paradigmesSwim2.columns.tolist()[25:]

In [ ]:
paradigmesSwim2.pc1S

# Calcul des distances

### Préparation des fichiers de référence
La comparaison se fait sur les lexèmes de *listeTest*
- initialForms : les formes initiales
- finalForms : les formes initiales plus les formes prédites
- goldForms : les formes de référence


#### Formes initiales

#### Formes avec prédictions

#### Formes de référence
- recoder les représentations phonologiques en fonction de l'échantillon

### Soustraction des formes initiales

### Évaluation des formes

#### Problèmes de génération
- sous-génération : formes de références sans prédiction
- sur-génération : formes prédites absentes de la référence

#### Formes comparables
- *predictedForms* => formes prédites présentes dans la référence
- *actualForms* => formes de référence présentes dans les prédictions

#### Préparation de la comparaison
- *compareForms* => tableau avec les formes à comparer

#### Comparaison des formes
- *sameForms* => formes identiques dans les prédictions et la référence
- *diffForms* => formes différentes dans les prédictions et la référence

#### Décompte du nombre de formes
À cause du traitement de la surabondance, le décompte des lignes ne tient pas compte du nombre de formes. Il faut compter les formes dans chaque ligne pour obtenir le nombre total.

#### Évaluation des différences
Certaines différences ne sont qu'apparentes à cause du traitement de la surabondance sous forme de chaine concaténée. Il faut transformer les surabondances en set() pour pouvoir faire une comparaison pertinente.

#### Calcul des grandeurs de mesure

### Sauvegarde des résultats pour analyse ultérieure

## Analyse des sur-générations

### Analyse par cases

Le facteur principal de **surgénération** est la production de **participes passés** (ppMP 662, ppFS 661, ppFP 661) pour les verbes qui n'ont qu'une seule forme de participe passé invariable (ppMS) : **54.78%** (1984/3622).

### Analyse par verbes

Le facteur principal de **surgénération** tient aux **verbes vestiges** et aux **météorologiques**.

## Analyse des sous-générations

### Analyse par cases

Le facteur principal de **sous-génération** est lié à la fréquence des TAM et de certaines personnes :
- TAM : 
 - imparfait du subjonctif 45.98% (7875/17126)
 - passé simple 12.70% (2176/17126)
- Personnes :
 - 2PL 46.11% (7897/17126)
 - 1PL 32.98% (5649/17126)


### Analyse par verbes

# Fin de l'analyse des résultats

### Test de la sauvegarde de l'analyse brute

## with open(analysisPrefix+"-analyseParadigmes.pkl","rb") as input:
    savedAnalyseParadigmes = pickle.load(input)    